In [11]:
import torch
torch.cuda.empty_cache()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
import random

import cv2, os, shutil
from matplotlib import pyplot as plt

In [3]:
# BOX_COLOR = (255, 0, 0) # Red
# TEXT_COLOR = (255, 255, 255) # White

# def visualize_bbox(img, bbox, class_name, pic_w, pic_h, color=BOX_COLOR, thickness=2):
#     dataType = "yolo"

#     """Visualizes a single bounding box on the image"""

#     if dataType == 'coco':
#         x_min, y_min, w, h = bbox # 정규화 된 0~1 사이의 값
#         x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
#     elif dataType == "yolo":
#         x_center, y_center, w, h = bbox
#         # 픽셀 좌표로 변환 이미지의 width와 height값
#         x_min = int(float(x_center - w/2) * pic_w)
#         x_max = int(float(x_center + w/2) * pic_w)
#         y_min = int(float(y_center - h/2) * pic_h)
#         y_max = int(float(y_center + h/2) * pic_h)
#     print(w, h)
#     print(x_min, y_min, y_min, y_max)
#     cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

#     ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
#     cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
#     cv2.putText(
#         img,
#         text=class_name,
#         org=(x_min, y_min - int(0.3 * text_height)),
#         fontFace=cv2.FONT_HERSHEY_SIMPLEX,
#         fontScale=0.35,
#         color=TEXT_COLOR,
#         lineType=cv2.LINE_AA,
#     )
#     return img


# def visualize(image, bboxes, category_ids, category_id_to_name, img_shape):
#     img = image.copy()
#     h,w = img_shape[0:2]
#     print(bboxes)
#     print(category_ids)
#     for bbox, category_id in zip(bboxes, category_ids):
#         class_name = category_id_to_name[category_id]
#         img = visualize_bbox(img, bbox, class_name, w, h)
#     plt.figure(figsize=(12, 12))
#     plt.axis('off')
#     plt.imshow(img)

In [4]:
# def read_label_txt(txtFile):
#     category_ids = []
#     bboxes = []

#     f=open(txtFile,'r')

#     while True:
#         line = f.readline()
#         if not line: break
#         ids, xc, yc, w, h= line.split(' ')
#         category_ids.append(int(ids))
#         bboxes.append([float(xc),float(yc),float(w),float(h)])
#         #print(line)
#     f.close()
#     return category_ids, bboxes

# def write_label_txt(txtFile, category_ids, bboxes):
#     f=open(txtFile,'w')

#     for i, ids in enumerate(category_ids):
#         xc,yc,w,h = bboxes[i]
#         f.write("{} {} {} {} {}\n".format(int(ids),xc,yc,w,h))
#         #print("{} {}".format(int(ids), bboxes[i]))
#     f.close()

In [5]:
# txtFile = 'sepImgLabSimpson/bart/bart_labels/pic_0000.txt'
# imagePath = 'sepImgLabSimpson/bart/bart_images/pic_0000.jpg'
# image = cv2.imread(imagePath)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# category_ids, bboxes = read_label_txt(txtFile)
# category_id_to_name = {0: 'bart', 1: 'homer', 2: 'lisa', 3: 'maggie', 4: 'marge'}
# visualize(image, bboxes, category_ids, category_id_to_name, image.shape[0:2])

In [1]:
trainRatio = 0.7
validRatio = 0.15
testRatio = 1 - trainRatio - validRatio

In [2]:
import os, shutil, random

# 데이터셋 디렉토리 경로 설정 (데이터셋이 저장된 경로로 수정하세요)
dataset_dir = os.path.join(os.getcwd(), 'sepImgLabSimpson')

# 새로운 train, valid, test 디렉토리 생성 경로
base_dir = os.path.join(os.getcwd(), 'sepImgLabSimpson_tvt')

# 클래스 목록
classes = ['bart', 'homer', 'lisa', 'maggie'] #marge
fileTypes = ['images','labels']

# 폴더 경로 생성
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# 폴더 생성 함수
def create_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

# train, valid, test 폴더 및 각각의 클래스 폴더 생성
# 각각의 클래스 폴더 및 이미지와 레이블 폴더 생성 
for fileType in fileTypes:
    create_dir(os.path.join(train_dir, fileType))
    create_dir(os.path.join(valid_dir, fileType))
    create_dir(os.path.join(test_dir, fileType))

def checkEqual(filesPath, label):
    images_path = os.path.join(filesPath, str(label + '_images'))
    labels_path = os.path.join(filesPath, str(label + '_labels'))
    imageFiles = os.listdir(images_path)
    labelFiles = os.listdir(labels_path)
    if len(imageFiles) != len(labelFiles):
        print(f'{label} imageFiles {len(imageFiles)} and labelFiles {len(labelFiles)} not equal')
        for count in range(len(labelFiles)):
            print(f'{count} imageFiles {imageFiles[count]} labelFiles {labelFiles[count]}')
        img_mismatch_list = [True] * len(imageFiles)
        label_mismatch_list = [True] * len(labelFiles)

        for count, image in enumerate(imageFiles):
            for label in labelFiles:
                if os.path.splitext(image)[0] == os.path.splitext(label)[0]:
                    img_mismatch_list[count] = False
                    break
        for count, label in enumerate(labelFiles):
            for image in imageFiles:
                if os.path.splitext(label)[0] == os.path.splitext(image)[0]:
                    label_mismatch_list[count] = False
                    break
                    
        for count in range(len(imageFiles)):
            if img_mismatch_list[count]: 
                print(f'no label for the following images: {imageFiles[count]}')
        for count in range(len(labelFiles)):
            if label_mismatch_list[count]: 
                print(f'no image for the following labels: {labelFiles[count]}')
        return False
    
    return True 

# def matchAndShuffle(filesPath):
#     img_dir = os.path.join(src_dir, str(label + '_images'))
#     label_dir = os.path.join(src_dir, str(label + '_labels'))
#     imageFiles = os.listdir( src_img_dir )
#     labelFiles = os.listdir( src_label_dir )
#     imageFiles = random.sample(imageFiles, len(imageFiles))
#     imageToLabel = dict.fromkeys(imageFiles)
#     for image in imageFiles:
#         for label in labelFiles:
#             if os.path.splitext(image) == os.path.splitext(label):
#                 imageToLabel[image] = label
#     print(imageToLabel)
    
#     return imageToLabel

# # 이미지 복사 함수
# def copy_images_and_labels(start_idx, end_idx, imageLabelDict, dst_dir, label):
#     for i in range(start_idx, end_idx + 1):
#         image_name = f'{label}.{i}.jpg'
#         label_name = f'{label}.{i}.txt'
#         src_imagePath = imageLabelDict[]
#         src_labelPath = os.path.join(src_label_dir, srcLabels[i])
#         dst_imagePath = os.path.join(dst_dir, 'images', image_name)
#         dst_labelPath = os.path.join(dst_dir, 'labels', label_name)
        
#         if os.path.exists(src_imagePath):
#             shutil.copy(src_imagePath, dst_imagePath)
#         if os.path.exists(src_labelPath):
#             shutil.copy(src_labelPath, dst_labelPath)

# 이미지 복사 함수
def copy_images_and_labels(start_idx, end_idx, src_dir, dst_dir, label):
    src_img_dir = os.path.join(src_dir, str(label + '_images'))
    src_label_dir = os.path.join(src_dir, str(label + '_labels'))
    srcImages = os.listdir( src_img_dir )
    srcLabels = os.listdir( src_label_dir )
    # print(f'srcImages1: {srcImages}')
    # print(f'srcLabels1: {srcLabels}')
    srcImages.sort()
    srcLabels.sort()
    # print(f'srcImages2: {srcImages}')
    # print(f'srcLabels2: {srcLabels}')
    for i in range(start_idx, end_idx + 1):
        image_name = f'{label}.{i}.jpg'
        label_name = f'{label}.{i}.txt'
        src_imagePath = os.path.join(src_img_dir, srcImages[i])
        src_labelPath = os.path.join(src_label_dir, srcLabels[i])
        dst_imagePath = os.path.join(dst_dir, 'images', image_name)
        dst_labelPath = os.path.join(dst_dir, 'labels', label_name)
        
        if os.path.exists(src_imagePath):
            shutil.copy(src_imagePath, dst_imagePath)
        if os.path.exists(src_labelPath):
            shutil.copy(src_labelPath, dst_labelPath)

# 클래스 별로 train, valid, test 데이터셋 구성
for cls in classes:
    #retrieve image count
    filesPath = os.path.join(os.getcwd(), 'sepImgLabSimpson', cls)

    if checkEqual(filesPath, cls):
    
        files = os.listdir( os.path.join(filesPath, str(cls + '_images')))
        
        train_num = int(trainRatio * len(files))
        valid_num = int(validRatio * len(files))
        test_num  = len(files) - train_num - valid_num 
    
        train_range = [0, train_num-1]
        valid_range = [train_num, train_num + valid_num -1]
        test_range  = [train_num + valid_num, train_num + valid_num + test_num-1]
        print(f'train {train_range} valid {valid_range} test {test_range}')

        # #create a dictionary matching label to image, then shuffle for equal distribution of samples in train, valid, and test folders each
        # imageLabelDict = matchAndShuffle(filesPath)
    
        #copy images and label
        copy_images_and_labels(train_range[0], train_range[1], os.path.join(dataset_dir, cls), os.path.join(train_dir), cls)
    
        # valid dataset 구성 (1000~1249)
        copy_images_and_labels(valid_range[0], valid_range[1], os.path.join(dataset_dir,cls), os.path.join(valid_dir), cls)
    
        # test dataset 구성 (1250~1499)
        copy_images_and_labels(test_range[0], test_range[1], os.path.join(dataset_dir,cls), os.path.join(test_dir), cls)

print("데이터셋 분할 및 복사가 완료되었습니다.")

bart imageFiles 1342 and labelFiles 1343 not equal
0 imageFiles pic_0000.jpg labelFiles .ipynb_checkpoints
1 imageFiles pic_0001.jpg labelFiles pic_0000.txt
2 imageFiles pic_0002.jpg labelFiles pic_0001.txt
3 imageFiles pic_0003.jpg labelFiles pic_0002.txt
4 imageFiles pic_0004.jpg labelFiles pic_0003.txt
5 imageFiles pic_0005.jpg labelFiles pic_0004.txt
6 imageFiles pic_0006.jpg labelFiles pic_0005.txt
7 imageFiles pic_0007.jpg labelFiles pic_0006.txt
8 imageFiles pic_0008.jpg labelFiles pic_0007.txt
9 imageFiles pic_0009.jpg labelFiles pic_0008.txt
10 imageFiles pic_0010.jpg labelFiles pic_0009.txt
11 imageFiles pic_0011.jpg labelFiles pic_0010.txt
12 imageFiles pic_0012.jpg labelFiles pic_0011.txt
13 imageFiles pic_0013.jpg labelFiles pic_0012.txt
14 imageFiles pic_0014.jpg labelFiles pic_0013.txt
15 imageFiles pic_0015.jpg labelFiles pic_0014.txt
16 imageFiles pic_0016.jpg labelFiles pic_0015.txt
17 imageFiles pic_0017.jpg labelFiles pic_0016.txt
18 imageFiles pic_0018.jpg labelFil

IndexError: list index out of range

In [13]:
from ultralytics import YOLO
model = YOLO('yolo11n.pt')

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
results = model.train(data='data.yaml', epochs=50, imgsz=320)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
